ДОКУМЕНТАЦИЯ!

функция принимает на вход numpy матрицу (like: <code>canonization(matrix)</code>)

note: можно написать <code>canonization(matrix, 1)</code> - разницы нет;
хотите сломать функцию? напишите <code>canonization(matrix, 0)</code>.
    
Функция выдает <code>[canonized_matrix, [new_rows_order, new_cols_order]]</code>, где:

корневой массив - list;

<tt>[new_rows_order, new_cols_order]</tt> - list

canonized_matrix, new_rows_order и new_cols_order - numpy.ndarray(s)

In [1]:
import itertools
import numpy as np
import collections
import copy

def canonization(a, is_root=1):
    rows_unordered_cols_ordered_separately = np.sort(a) # ordered columns in each row separately, but no rows

    str_rows_unordered_list = []
    for i in range(rows_unordered_cols_ordered_separately.shape[0]): 
        str_rows_unordered_list.append(''.join(str(e) for e in rows_unordered_cols_ordered_separately[i]))

    rows_order = np.argsort(str_rows_unordered_list)
    rows_ordered = np.empty((0, a.shape[1]), int)
    for i in rows_order:
        rows_ordered = np.concatenate((rows_ordered, [a[i]]))

    # тут столбцы
    rows_ordered_separately = np.sort(rows_ordered, axis = 0) # ordered columns in each row separately, but no rows

    str_cols_unordered_list = []
    for i in range(rows_ordered_separately.shape[1]):
        str_cols_unordered_list.append(''.join(str(col) for col in rows_ordered_separately[:, i]))


    cols_order = np.argsort(str_cols_unordered_list)

    cols_ordered = np.empty((a.shape[0], 0), int) # it will really cols_ordered_rows_ordered
    for i in cols_order:
        cols_ordered = np.concatenate((cols_ordered, rows_ordered[:, i:i+1]), axis=1)



    # тут собираем более подробную информацию про разбиение на группы

    groups_tmp = collections.Counter()
    rows_group_division = []
    for row_index in rows_order:
        groups_tmp[str_rows_unordered_list[row_index]] += 1
        if (groups_tmp[str_rows_unordered_list[row_index]] == 1):
            rows_group_division.append(1)
        else:
            rows_group_division[-1] += 1

    groups_tmp = collections.Counter()

    cols_group_division = []
    for col_index in cols_order:
        groups_tmp[str_cols_unordered_list[col_index]] += 1
        if (groups_tmp[str_cols_unordered_list[col_index]] == 1):
            cols_group_division.append(1)
        else:
            cols_group_division[-1] += 1


    group_division = []
    cur_row = cur_col = 0
    max_group_count = max(len(cols_group_division), len(rows_group_division))
    for i in range(max_group_count):
        rows_count = rows_group_division[-1]
        cols_count = cols_group_division[-1]
        if i < len(rows_group_division):
            rows_count = rows_group_division[i]
            cur_row += rows_count
        if i < len(cols_group_division):
            cols_count = cols_group_division[i]
            cur_col += cols_count
        group_division.append(((cur_row - rows_count, cur_row), (cur_col - cols_count, cur_col)))

    permutations = [rows_order, cols_order]

    group_sorted_matrix = cols_ordered
    if is_root:

        for indices in group_division:
            submatrix = group_sorted_matrix[indices[0][0]:indices[0][1],indices[1][0]:indices[1][1]]
            [tmp_submatrix, group_permutation] = canonization(submatrix, 0)

            tmp_subrows = np.copy(group_sorted_matrix[indices[0][0]:indices[0][1], :])
            tmp_subRowPemutation = np.copy(rows_order[indices[0][0]:indices[0][1]])

            for row_index in range(len(tmp_subrows)):
                group_sorted_matrix[indices[0][0] + row_index] = tmp_subrows[group_permutation[0][row_index]]
                rows_order[indices[0][0] + row_index] = tmp_subRowPemutation[group_permutation[0][row_index]]
        
            tmp_subcols = np.copy(group_sorted_matrix[:, indices[1][0]:indices[1][1]])
            
            tmp_subColPemutation = np.copy(cols_order[indices[1][0]:indices[1][1]])

            for col_index in range(tmp_subcols.shape[1]):
                group_sorted_matrix[:, indices[1][0] + col_index:indices[1][0] + col_index + 1] = tmp_subcols[:, group_permutation[1][col_index]:group_permutation[1][col_index]+1]
                cols_order[indices[1][0] + col_index] = tmp_subColPemutation[group_permutation[1][col_index]]

    return [group_sorted_matrix, permutations]